In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from chains.vision_assistant import VisionAssistant
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain_core.messages import HumanMessage
from PIL import ImageOps, Image
import base64
import io
from dotenv import load_dotenv
import os

from utils import image_processor 

# Load environment variables from .env file
load_dotenv()
phi_model = VisionAssistant
# Initialize NVIDIA models
# phi_model = ChatNVIDIA(model="microsoft/phi-3-vision-128k-instruct", base_url="https://integrate.api.nvidia.com/v1")
# granite_model = ChatNVIDIA(model="ibm/granite-34b-code-instruct", base_url="https://integrate.api.nvidia.com/v1")
# llama_model = ChatNVIDIA(model="meta/llama3-70b-instruct", base_url="https://integrate.api.nvidia.com/v1")

@tool
def get_recipe_from_ingredients(img_path: str) -> str:
    """Returns a list of edible ingredients that are in an image."""
    image_base64 = image_processor.resize_image_to_base64(img_path)
    text_input = f"""Thoroughly analyze the image and provide a list of edible ingredients from the image.
    """

    prompt = HumanMessage(content=[
        {"type": "text", "text": text_input},
        {"type": "image_url", "image_url": f"data:image/png;base64,{image_base64}"}
    ])
    result = phi_model.invoke([prompt])
    return result.content
    
    return result.content




# UML Descriptor

In [31]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain_core.messages import HumanMessage
from PIL import ImageOps, Image
import base64
import io
from dotenv import load_dotenv
import os

from utils import image_processor 

# Load environment variables from .env file
load_dotenv()

# Initialize NVIDIA models
phi_model = ChatNVIDIA(model="microsoft/phi-3-vision-128k-instruct", base_url="https://integrate.api.nvidia.com/v1")
granite_model = ChatNVIDIA(model="ibm/granite-34b-code-instruct", base_url="https://integrate.api.nvidia.com/v1")
llama_model = ChatNVIDIA(model="meta/llama3-70b-instruct", base_url="https://integrate.api.nvidia.com/v1")

@tool
def get_uml_description(img_path: str) -> str:
    """Returns the UML description of the image."""
    image_base64 = image_processor.resize_image_to_base64(img_path)
    text_input = """ You are an experienced software engineer who specializes in analyzing UML diagrams and translating them into detailed descriptions. Your task is to thoroughly analyze UML diagrams provide a structured description. Be sure to include all the class names, attributes, methods, and relationships.
        Example:
        Class Name: Person
            - Attributes:
                - name: String
                - phoneNumber: String
                - emailAddress: String
            - Methods:
                - purchaseParkingPass(): void

        Class Name: Student
            - Attributes:
                - studentNumber: Integer
                - averageMark: Integer
            - Methods:
                - isEligibleToEnroll(course: String): boolean
                - getSeminarsTaken(): Integer
        
        Class Name: Professor
            - Attributes:
                - salary: Integer
                - staffNumber: Integer
                - yearsOfService: Integer
                - numberOfClasses: Integer
            - Methods:
                - None
        
        Class Name: Address
            - Attributes:
                - street: String
                - city: String
                - state: String
                - postalCode: Integer
                - country: String
            - Methods:
                - validate(): boolean
                - outputAsLabel(): String
        
        Relationships:
            - Inheritance:
                - Student and Professor inherit from Person.
            - Association:
                - Person - Address:
                    - A Person lives at an Address.
                    - Multiplicity: 0..1 (Person) to 1 (Address)
                - Professor - Student:
                    - A Professor supervises Students.
                    - Multiplicity: 1..5 (Professor) to 0..* (Student)
                        """

    prompt = HumanMessage(content=[
        {"type": "text", "text": text_input},
        {"type": "image_url", "image_url": f"data:image/png;base64,{image_base64}"}
    ])
    result = phi_model.invoke([prompt])
    return result.content


@tool
def get_code_from_description(description: str) -> str:
    """
    Returns the code generated from the provided UML description.

    Args:
        description (str): The structured description of a UML diagram.

    Returns:
        str: The corresponding code generated from the UML description.
    """
    try:
        # Define the text input for the model
        text_input = f"""
        Using the following description of a UML diagram, write the corresponding code using classes. 
        Here is the description: {description}
        """
        
        # Create the prompt
        prompt = HumanMessage(content=text_input)
        
        # Invoke the model
        result = granite_model.invoke([prompt])
        
        return result.content
    
    except Exception as e:
        return f"An error occurred: {str(e)}"



# Test

In [32]:
# Register tools
tools = [get_uml_description, get_code_from_description, get_recipe_from_ingredients]

try:
    # Initialize the agent with the registered tools
    agent = initialize_agent(
        tools=tools,
        llm=llama_model,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )
    print("Agent initialized successfully.")
except Exception as e:
    print(f"An error occurred during agent initialization: {str(e)}")


Agent initialized successfully.


### UML to Code

In [33]:
import os

def main():
    """
    Main function to generate code based on a UML diagram image.
    """
    # Path to the UML diagram image
    uml_image_path = '/Users/ganesh/Downloads/LMS-Class-Diagram.jpg'
    
    # Ensure the path string is clean and without extra newline characters
    uml_image_path = uml_image_path.strip()

    # Check if the file exists
    if not os.path.exists(uml_image_path):
        print(f"File not found: {uml_image_path}")
        return

    try:
        # Get UML description using the agent
        uml_description = agent.run(f"Generate Java code based on the following UML diagram: {uml_image_path}")
        
        # Print the UML description
        print("UML Description:")
        print(uml_description)
    
    except Exception as e:
        print(f"An error occurred while generating UML description: {str(e)}")

if __name__ == "__main__":
    main()




> Entering new AgentExecutor chain...
I need to generate Java code based on a UML diagram image.

Action: get_uml_description
Action Input: /Users/ganesh/Downloads/LMS-Class-Diagram.jpg
Observation: Title: Library Management System

Classes:
1. Book
    - Attributes:
        - bookId: Integer
        - title: String
        - author: String
        - ISBN: String
        - publisher: String
        - price: Integer
        - edition: Integer
        - isAvailable: Boolean
    - Methods:
        - addBook(book: Book): void
        - removeBook(book: Book): void
        - updateBook(book: Book): void
        - searchBook(searchTerm: String): List<Book>
        - getBookDetails(bookId: Integer): Book
        - getBooksByAuthor(author: String): List<Book>
        - getBooksByPrice(price: Integer): List<Book>
        - getBooksByISBN(isbn: String): List<Book>
        - getBooksByPublisher(publisher: String): List<Book>
        - getBooksByEdition(edition: Integer): List<Book>
        - ge

KeyboardInterrupt: 

In [ ]:
import os

def main():
    food_img = '/Users/ganesh/Desktop/food.png'
    
    # Ensure the path string is clean and without extra newline characters
    food_img = food_img.strip()

    # Check if the file exists
    if not os.path.exists(food_img):
        print(f"File not found: {food_img}")
        return

    try:
        # Ensure the agent is defined somewhere in your code
        # Get a recipe using the agent
        recipe = agent.run(f"Can you give me an Italian recipe that I can make from the ingredients in this image: {food_img}")
        
        # Print the recipe
        print("Italian Recipe:")
        print(recipe)
    
    except Exception as e:
        print(f"An error occurred while generating the recipe: {str(e)}")

if __name__ == "__main__":
    main()


In [27]:
from langchain_core.tools import StructuredTool

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# Create structured tools for both sync and async implementations
sync_tool = StructuredTool.from_function(multiply)
async_tool = StructuredTool.from_function(amultiply)


In [28]:
from langchain_core.tools import BaseTool
from typing import Optional, Union
from math import sqrt, cos, sin

class PythagorasTool(BaseTool):
    name = "Hypotenuse calculator"
    description = (
        "Calculate the length of a hypotenuse given one or two sides of a triangle "
        "and/or an angle (in degrees). Requires at least two of the following: "
        "adjacent_side, opposite_side, angle."
    )

    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Need two or more of `adjacent_side`, `opposite_side`, or `angle`."

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


In [30]:
from langchain.chains.llm import LLMChain
from langchain.agents import AgentExecutor
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

tools = [PythagorasTool()]

# Create a prompt template
template = """
Question: {input}
Thought: {agent_scratchpad}
Action: {action}
Action Input: {action_input}
"""

prompt = PromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

# Set up the LLM chain and the agent executor
llm = OpenAI()
llm_chain = LLMChain(llm=llm, prompt=prompt)
executor = AgentExecutor(agent=llm_chain, tools=tools)

# Run the agent with an input
result = executor.invoke({"input": "What is the hypotenuse of a triangle with sides 3 and 4?"})
print(result.output)  # Outputs: 5.0


/Users/ganesh/miniconda3/envs/nim/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


ValueError: Missing some input keys: {'action', 'action_input', 'agent_scratchpad'}